<a href="https://colab.research.google.com/github/vardansaini/Custom-T5/blob/main/T5_Glove_HuggingFace_custom_class_bulid_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install SentencePiece
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from transformers import EncoderDecoderConfig, EncoderDecoderModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration, T5Tokenizer
import torchtext
import os
import pathlib

from numpy import random
from scipy import spatial

import torch.nn as nn
from torch.optim import Adam

import torch

import csv

dims = 50

from torchtext.vocab import GloVe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
glove = ('/content/drive/MyDrive/glove.6B.50d.txt')

In [ ]:
# Reading glove text file
with open(glove) as f:
    lines = f.readlines()

glove_dict = {}

# converting in key value format
# example ("the" : ["0.22323", "0.0023232", ......])
for i in lines:
  tokens  = i.split(" ")
  key = tokens[0]
  tokens.pop(0)
  value = tokens
  glove_dict[key] = value

print(len(glove_dict))

400000


In [ ]:
v = glove_dict['the']

float_glove_dict = {}

# converting values from string to float and muliplying to convert from floats to long/int later.
for key, value in glove_dict.items():
  l = []
  for i in value:
    f = float(i) * 10000
    l.append(f)
  float_glove_dict[key] = l

In [ ]:
# # converting to tensor

# count = 0
# tens = []
# for embeddings in float_glove_dict.values():
#   print(embeddings)
#   j = torch.IntTensor(embeddings)
#   print(j[0])
#   j = j.unsqueeze(0)
#   tens.append(j)
#   if count == 2:
#     break
#   count += 1

# # print(tens[0] * 10)



# print(tens)
# print(type(tens))
# print(len(tens))

# converting to tensor

count = 0
tens = []
for embeddings in float_glove_dict.values():
  print(embeddings)
  # j = torch.IntTensor(embeddings)
  # print(j[0])
  # j = j.unsqueeze(0)
  # tens.append(j)
  int_embd = [int(i) for i in embeddings]
  j = tf.constant(int_embd)
  print(j[0].numpy())

  j = tf.expand_dims(j, axis=0)
  tens.append(j)
  if count == 2:
    break
  count += 1

# print(tens[0] * 10)



print(tens)
print(type(tens))
print(len(tens))

[4180.0, 2496.8, -4124.2, 1217.0, 3452.7000000000003, -444.57000000000005, -4968.8, -1786.2, -6.6023000000000005, -6566.0, 2784.3, -1476.7, -5567.7, 1465.8, -95.095, 116.58, 1020.4000000000001, -1279.2, -8443.0, -1218.1, -168.01, -3327.8999999999996, -1552.0, -2313.1, -1918.1000000000001, -18823.0, -7674.6, 990.51, -4212.5, -1952.6, 40071.0, -1859.3999999999999, -5228.7, -3168.1, 5.9213, 74.449, 1777.8, -1589.7, 120.41, -542.23, -2987.1, -1574.8999999999999, -3475.8, -456.36999999999995, -4425.1, 1878.5, 27.849, -1841.1, -1151.4, -7858.1]
4180
[134.41, 2368.2, -1689.9, 4095.1, 6381.2, 4770.900000000001, -4285.2, -5564.099999999999, -3640.0, -2393.8, 1300.1, -637.34, -3957.5, -4816.2, 2329.1, 902.01, -1332.3999999999999, 786.39, -4163.4, -1542.8, 1006.8000000000001, 4889.1, 3122.6, -1252.0, -375.11999999999995, -15179.0, 1261.2, -244.20000000000002, -429.61, -2835.1, 35416.0, -1195.6, -145.33, -1499.0, 2186.4, -3341.2, -1387.2, 3180.6, 7035.8, 4485.8, -802.62, 6300.3, 3211.1, -4676.5, 2

In [ ]:
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.", return_tensors="pt")
print(encoded_input)

{'input_ids': tensor([[  531,    59,   140,  8437,    16,     8, 12748,    13, 25027,     7,
             6,    21,    79,    33,  9240,    11,  1704,    12, 11213,     5,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
decoded_input = tokenizer("What about elevensies?",  return_tensors="pt")
print(decoded_input)

{'input_ids': tensor([[  363,    81, 20394,     7,   725,    58,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
input_ids = encoded_input.input_ids
# input_ids = tens[0]
# print(torch.max(input_ids))
# print(torch.max(input_ids, dim=0))
# input_ids = input_ids.unsqueeze(0)
print(input_ids.shape)
# print(input_ids.unsqueeze(0))
print(input_ids)
print(type(input_ids))
print(len(input_ids[0]))
# attention_mask = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

torch.Size([1, 21])
tensor([[  531,    59,   140,  8437,    16,     8, 12748,    13, 25027,     7,
             6,    21,    79,    33,  9240,    11,  1704,    12, 11213,     5,
             1]])
<class 'torch.Tensor'>
21


In [ ]:
decoder_input_ids = decoded_input.input_ids

decoder_input_ids = tens[1]
# print(decoder_input_ids.shape)
print(decoder_input_ids)
print(type(decoder_input_ids))
print(len(decoder_input_ids[0]))
# decoder_attention_mask = [1, 1, 1, 1, 1, 1, 1]

tf.Tensor(
[[   134   2368  -1689   4095   6381   4770  -4285  -5564  -3640  -2393
    1300   -637  -3957  -4816   2329    902  -1332    786  -4163  -1542
    1006   4889   3122  -1252   -375 -15179   1261   -244   -429  -2835
   35416  -1195   -145  -1499   2186  -3341  -1387   3180   7035   4485
    -802   6300   3211  -4676   2278   3603  -3781  -5665    446   3039]], shape=(1, 50), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
50


In [ ]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
# class CustomModel(nn.Module):
#   def __init__(self,input_ids, decoder_input_ids):
#     super(CustomModel,self).__init__()
#     # self.input_embeddings = input_ids

#     #Load Model with given checkpoint and extract its body
#     self.model = model
#     self.dropout = nn.Dropout(0.1)
#     self.emdedding_layer = nn.Linear(50) # load and initialize weights

#   def forward(self, input_ids=None, attention_mask=None):
#           embedding_output = self.model(input_ids=input_ids, attention_mask=attention_mask)
#           sequence_output = self.dropout(embedding_output[0]) #outputs[0]=last hidden state

#           # sequence_output has the following shape: (batch_size, sequence_length, 768)
#           linear1_output = self.linear1(sequence_output[:,0,:].view(1,50)) ## extract the 1st token's embeddings

#           linear2_output = self.emdedding_layer(linear1_output)

#           return linear2_output

In [ ]:
class CustomModel(nn.Module):
  def __init__(self,num_labels):
    super(CustomModel,self).__init__()
    self.num_labels = num_labels

    #Load Model with given checkpoint and extract its body
    self.model = model
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768,num_labels) # load and initialize weights

  def forward(self, input_ids=None, attention_mask=None,labels=None):
    #Extract outputs from the body
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

    #Add custom layers
    sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state

    logits = self.classifier(sequence_output[:,0,:].view(-1,768)) # calculate losses

    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [ ]:
outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
# print(outputs)

output = model.generate(input_ids=input_ids)

print(output)

output_str = tokenizer.batch_decode(output, skip_special_tokens=True)

print(output_str)

AttributeError: ignored